# Algoritmos de clasificación Proyecto Integrador 2

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


### Importar librerías

In [40]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency # para ver relación entre y y variables categóricas
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree  import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix, classification_report, roc_auc_score
from sklearn.svm import SVC
from pprint import pprint
from subprocess import check_output
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
from collections import OrderedDict

%matplotlib inline

In [5]:
df= pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/PI2/data_pca.csv')

In [6]:
index=df['Customer_ID']
X = df.iloc[:,1:-1]
y = df.iloc[:,28]

In [7]:
X.head()

,Z1,Z2,Z3,Z4,Z5,Z6,Z7,Z8,Z9,Z10,...,Z18,Z19,Z20,Z21,Z22,Z23,Z24,Z25,Z26,Z27
0,-1.432269,0.093852,-2.278188,-1.407816,-0.135649,-1.996803,0.275828,-0.648048,1.313756,-0.713133,...,0.575968,0.667774,0.094029,-0.093014,-1.100186,-1.494958,1.076635,-1.688978,0.175915,-0.819142
1,-1.698989,0.418354,-1.785871,-1.361343,-0.131089,-1.898032,0.406715,-0.559605,1.350804,-0.727694,...,0.592985,0.595757,-0.238172,0.142565,-1.179211,-1.534768,0.976853,-1.743069,0.064623,-0.666969
2,-1.501422,0.479586,-1.432461,-1.360819,-0.066890,-1.854806,0.282160,-0.416728,1.355014,-0.817413,...,0.454585,1.041039,0.708216,-0.509633,-1.109486,-1.522738,1.003116,-1.690114,0.156681,-0.533545
3,-1.682258,0.101027,-2.322815,-1.393282,-0.149784,-1.995335,0.318837,-0.708591,1.318216,-0.665006,...,0.666473,0.514263,-0.515451,0.413239,-1.106747,-1.530995,0.928847,-1.573458,0.124982,-0.749807
4,-1.582761,0.624981,-1.505005,-1.382160,-0.070191,-1.849546,0.218287,-0.380822,1.363299,-0.871936,...,0.384913,1.189984,1.238533,-0.868051,-1.067568,-1.493789,1.074852,-1.702390,0.253632,-0.557135


In [8]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=2, test_size=0.3, random_state=0)
for train_index, test_index in gss.split(X, groups=index):
    print("Train:", train_index, "Test:", test_index)
    X_train = X.iloc[train_index, :]
    y_train = y[train_index]
    X_test = X.iloc[test_index, :]
    y_test = y[test_index]
    

Train: [    0     1     2 ... 99989 99990 99991] Test: [   16    17    18 ... 99997 99998 99999]
Train: [    0     1     2 ... 99997 99998 99999] Test: [    8     9    10 ... 99949 99950 99951]


### LGBM

In [46]:
LGBM = lgb.LGBMClassifier(random_state=42)
LGBM.fit(X_train,y_train)
y_pred = LGBM.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[ 5118  2891   664]
 [ 2564 11841  1788]
 [   98  2546  2490]]
              precision    recall  f1-score   support

           0    0.59011   0.65784   0.62214      7780
           1    0.73124   0.68532   0.70754     17278
           2    0.48500   0.50384   0.49424      4942

    accuracy                        0.64830     30000
   macro avg    0.60212   0.61567   0.60797     30000
weighted avg    0.65408   0.64830   0.65025     30000



In [47]:
LGBM2 = lgb.LGBMClassifier(random_state=42,n_jobs=8,max_depth=15,n_estimators=500,objective='multiclass',class_weight='balanced',learning_rate=0.01)
LGBM2.fit(X_train,y_train)
y_pred = LGBM2.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_pred,y_test,digits=5))

[[6122 1279 1272]
 [3714 8574 3905]
 [ 184 1128 3822]]
              precision    recall  f1-score   support

           0    0.70587   0.61098   0.65500     10020
           1    0.52949   0.78080   0.63104     10981
           2    0.74445   0.42471   0.54086      8999

    accuracy                        0.61727     30000
   macro avg    0.65994   0.60550   0.60897     30000
weighted avg    0.65288   0.61727   0.61200     30000



In [48]:
CVSLGBM2 = cross_val_score(LGBM2, X_train,y_train, cv=5,scoring='f1_macro')
print([CVSLGBM2,np.average(CVSLGBM2)])

[array([0.61584002, 0.6090957 , 0.6065875 , 0.61181131, 0.61872391]), 0.6124116865551537]
